<div style="border-radius: 15px 0 15px 0px; border: 2px solid #f1c40f; padding: 10px; background-color: #191970; text-align: center; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.2);">
    <h1 style="color: #f1c40f; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 24px;">Makine Öğrenmesiyle Öğrencilerin Yaşam Alışkanlıkları ve Akademik Durumlarına Göre Depresyon Riskinin Tahmini</h1>
</div>

# **1) Proje Tanımı**

        Amaç, öğrencilerin yaşam tarzı alışkanlıkları (uyku, sosyal aktivite, stres, fiziksel aktivite, çalışma saati vb.) ve akademik performans  göstergeleri (CGPA, ders çalışma saatleri, akademik tatmin) kullanılarak depresyon yaşayıp yaşamadıklarını tahmin eden bir makine öğrenmesi modeli geliştirmektir.

# **2) Veri Kaynakları ve Birleştirme Stratejisi**

## **2.1) Dataset 1** : Daily Lifestyle and Academic Performance

- 2.000 öğrenciden Google Form anketiyle toplanan verileri içerir.

- Ders çalışma süresi, sosyal etkinlikler, uyku, sosyalleşme, fiziksel aktivite, stres düzeyi ve not ortalaması bilgilerini kapsar.

- Veriler, Ağustos 2023 - Mayıs 2024 akademik yılına aittir.

- Katılımcıların çoğu Hindistan’daki öğrencilerden oluşmaktadır.

- **Amaç:** Günlük alışkanlıkların akademik başarı ve öğrenci refahı üzerindeki etkisini incelemek.

## 2.2) Dataset 2 : Student Mental Health

- Öğrencilerin ruh sağlığı ve depresyon riskini etkileyen faktörleri kapsamlı şekilde içerir.

- Demografik bilgiler, akademik ve iş baskıları, yaşam tarzı alışkanlıkları ve ruh sağlığı göstergelerini kapsar.

- **Amaç:** Depresyon eğilimlerini ve risk faktörlerini analiz etmek.

# **3) Yüklemeler ve İlk Kontroller**

### Paketlerin Yüklenmesi

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### Veri Yükleme

In [17]:
ds1 = pd.read_csv("./datasets/student_lifestyle_dataset.csv")
ds2 = pd.read_csv("./datasets/student_depression_dataset.csv")

### Temel Veri Bilgileri

In [58]:
print("\nDataset 1")
print(f"Shape (rows, columns): {ds1.shape}")
print("\nFirst 5 rows:")
display(ds1.head())

print("\n\n")

print("\nDataset 2")
print(f"Shape (rows, columns): {ds2.shape}")
print("\nFirst 5 rows:")
display(ds2.head())


Dataset 1
Shape (rows, columns): (2000, 8)

First 5 rows:


,Student_ID,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,GPA,Stress_Level
0,1,6.9,3.8,8.7,2.8,1.8,2.99,Moderate
1,2,5.3,3.5,8.0,4.2,3.0,2.75,Low
2,3,5.1,3.9,9.2,1.2,4.6,2.67,Low
3,4,6.5,2.1,7.2,1.7,6.5,2.88,Moderate
4,5,8.1,0.6,6.5,2.2,6.6,3.51,High






Dataset 2
Shape (rows, columns): (27901, 18)

First 5 rows:


,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0


### Genel Bilgi

In [59]:
print("\nDataset 1")
display(ds1.info())

print("\n\n")

print("\nDataset 2")
display(ds2.info())


Dataset 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Student_ID                       2000 non-null   int64  
 1   Study_Hours_Per_Day              2000 non-null   float64
 2   Extracurricular_Hours_Per_Day    2000 non-null   float64
 3   Sleep_Hours_Per_Day              2000 non-null   float64
 4   Social_Hours_Per_Day             2000 non-null   float64
 5   Physical_Activity_Hours_Per_Day  2000 non-null   float64
 6   GPA                              2000 non-null   float64
 7   Stress_Level                     2000 non-null   object 
dtypes: float64(6), int64(1), object(1)
memory usage: 125.1+ KB


None





Dataset 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  o

None

### İstatiksel Özeti

#### Dataset 1

In [60]:
display(ds1.drop(['Student_ID'],axis=1).describe().T)

,count,mean,std,min,25%,50%,75%,max
Study_Hours_Per_Day,2000.0,7.47580,1.423888,5.00,6.3,7.40,8.70,10.0
Extracurricular_Hours_Per_Day,2000.0,1.99010,1.155855,0.00,1.0,2.00,3.00,4.0
Sleep_Hours_Per_Day,2000.0,7.50125,1.460949,5.00,6.2,7.50,8.80,10.0
Social_Hours_Per_Day,2000.0,2.70455,1.688514,0.00,1.2,2.60,4.10,6.0
Physical_Activity_Hours_Per_Day,2000.0,4.32830,2.514110,0.00,2.4,4.10,6.10,13.0
GPA,2000.0,3.11596,0.298674,2.24,2.9,3.11,3.33,4.0


1. **Veri Kümesi Boyutları**:
   - Satir: 2,000
   - Sütun: 8
   - Sütunların hiçbirinde eksik değere rastlanmadı.

2. **Veri Türleri**:
   - Veri kümesi şunları içerir:
     - Sayısal sütunlar: 6 (örneğin, aktivitelere harcanan saatler ve not ortalaması).
     - Kategorik sütun: 1 (Stres Düzeyi).
     - ID Sütunu: 1 (Student_ID).

#### **İlk Gözlemler**
1. **Dengeli Yaşam Tarzı**:
   - Öğrencilerin ders çalışmaya, uykuya ve fiziksel aktiviteye harcadıkları ortalama saatlerin birbirine yakın olması, öğrenciler arasında nispeten dengeli bir yaşam tarzının olduğunu göstermektedir.

2. **GPA ve Yaşam Tarzı Korelasyonu**:
   - Nispeten yüksek ortalama not ortalaması, dengeli yaşam tarzı alışkanlıkları (örneğin yeterli uyku ve çalışma) ile akademik performans arasında olası pozitif bir ilişki olduğunu düşündürmektedir.

3. **Stres Seviyeleri**:
   - Kategorik bir özellik olarak, bunun GPA ve diğer değişkenlerle korelasyonları daha detaylı incelenebilir.

4. **Aykırı Değerler**:
   - Fiziksel aktivite saatleri (maksimum: 13 saat) bazı öğrencilerin olağanüstü derecede aktif olduğunu göstermektedir.
   - Sosyalleşme saatleri de değişkendir ve günlük 6 saate kadar çıkabilir, bu da çeşitli sosyal alışkanlıkların göstergesidir.

#### Dataset 2

In [61]:
display(ds2.drop(['id','Depression'],axis=1).describe().T)

,count,mean,std,min,25%,50%,75%,max
Age,27901.0,25.822300,4.905687,18.0,21.00,25.00,30.00,59.0
Academic Pressure,27901.0,3.141214,1.381465,0.0,2.00,3.00,4.00,5.0
Work Pressure,27901.0,0.000430,0.043992,0.0,0.00,0.00,0.00,5.0
CGPA,27901.0,7.656104,1.470707,0.0,6.29,7.77,8.92,10.0
Study Satisfaction,27901.0,2.943837,1.361148,0.0,2.00,3.00,4.00,5.0
Job Satisfaction,27901.0,0.000681,0.044394,0.0,0.00,0.00,0.00,4.0
Work/Study Hours,27901.0,7.156984,3.707642,0.0,4.00,8.00,10.00,12.0


1. **Veri Kümesi Boyutları**
    - Satır: 27,901
    - Sütun: 9
    - Eksik Değer: Hiçbir sütunda eksik değer bulunmamaktadır.

2. **Veri Türleri**
   - Sayısal sütunlar: 9 (yaş, akademik baskı, iş baskısı, CGPA, çalışma memnuniyeti, iş memnuniyeti, çalışma/öğrenim saatleri vb.)
   - Kategorik sütunlar: 8 (cinsiyet,şehir,beslenme alışkanlıkları, family history vb.)
   - ID Sütunu: 1 (ID)

#### **İlk Gözlemler**

1. **Demografik Dağılım (Age)**
   - Ortalama yaş 25.82, minimum 18, maksimum 59 olarak gözlenmiştir.
   - Bu, veri kümesinde hem üniversite öğrencilerini hem de mezun/çalışan bireyleri içeren geniş bir yaş aralığına işaret etmektedir.

2. **Akademik Baskı ve CGPA**
   - Akademik Baskı (mean = 3.14) orta seviyededir.
   - CGPA ortalaması 7.65 olup, bu değer oldukça iyi bir akademik performansa işaret eder.
   - Bu iki değişkenin birlikte incelenmesi, akademik baskının başarı üzerindeki etkisini anlamada kritik olacaktır.

3. **Çalışma Memnuniyeti (Study Satisfaction)**
   - Ortalama 2.94, medyan 3.0, bu da memnuniyetin orta düzeyde olduğunu gösterir.
   - Akademik baskı ve çalışma memnuniyeti birlikte analiz edildiğinde, baskı arttıkça memnuniyetin azalıp azalmadığı değerlendirilebilir.

4. **Aykırı Değerler**
   - Work Pressure ve Job Satisfaction değişkenlerinde 5 ve 4 değerlerine kadar çıkan seyrek yüksek değerler, aykırı gözlemler olabilir.
   - Age değişkeninde 59 yaşa kadar çıkılması da, bazı katılımcıların öğrenci dışı profilde olabileceğini düşündürmektedir. 

In [3]:
import os

In [4]:
print("files in current directory")
print(os.listdir())

files in current directory
['.git', '.gitignore', '.ipynb_checkpoints', 'anaconda_projects', 'datasets', 'env', 'UniMind.ipynb']


# **İçindekiler**

## 1. Proje Tanımı & Hedefler
## 2. Veri Kaynakları ve Birleştirme Stratejisi
## 3. Yüklemeler ve İlk Kontroller
## 4. Keşifsel Veri Analizi (EDA)
## 5. Veri Ön İşleme
## 6. Özellik Mühendisliği
## 7. Model Seçimi ve Eğitim (en az 3 model)
## 8. Model Değerlendirme ve Karşılaştırma
## 9. Model Optimizasyonu (Hyperparametre Tuning)
## 10. Açıklanabilirlik (Feature importance, SHAP)
## 11. Sonuçlar ve Raporlama / Sunum İçeriği
## 12. Ek Notlar: Etik, Veri Gizliliği, Kaynakça